In [1]:
#face and age Detection

# Mohamed Awadallah Bakir && Amr Ftouh abbas

import cv2 as cv
import math
import time


def faceDetection(net, frame, conf_threshold=0.7):
    frameOpencvDnn = frame.copy()
    frameHeight = frameOpencvDnn.shape[0]
    frameWidth = frameOpencvDnn.shape[1]
    blob = cv.dnn.blobFromImage(frameOpencvDnn, 1.0, (300, 300), [104, 117, 123], True, False)

    net.setInput(blob)
    detections = net.forward()
    bboxes = []
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > conf_threshold:
            x1 = int(detections[0, 0, i, 3] * frameWidth)
            y1 = int(detections[0, 0, i, 4] * frameHeight)
            x2 = int(detections[0, 0, i, 5] * frameWidth)
            y2 = int(detections[0, 0, i, 6] * frameHeight)
            bboxes.append([x1, y1, x2, y2])
            cv.rectangle(frameOpencvDnn, (x1, y1), (x2, y2), (0, 255, 0), int(round(frameHeight/150)), 8)
    return frameOpencvDnn, bboxes

faceProto = "modelNweight/opencv_face_detector.pbtxt"
faceModel = "modelNweight/opencv_face_detector_uint8.pb"

ageProto = "modelNweight/age_deploy.prototxt"
ageModel = "modelNweight/age_net.caffemodel"

genderProto = "modelNweight/gender_deploy.prototxt"
genderModel = "modelNweight/gender_net.caffemodel"

MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
ageList = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']
genderList = ['Male', 'Female']

ageNet = cv.dnn.readNet(ageModel, ageProto)
genderNet = cv.dnn.readNet(genderModel, genderProto)
faceNet = cv.dnn.readNet(faceModel, faceProto)

padding = 20

def Age_Gender_Pred(frame):
    t = time.time()
    frameFace, bboxes = faceDetection(faceNet, frame)
    for bbox in bboxes:
        face = frame[max(0,bbox[1]-padding):min(bbox[3]+padding,frame.shape[0]-1),max(0,bbox[0]-padding):min(bbox[2]+padding, frame.shape[1]-1)]

        blob = cv.dnn.blobFromImage(face, 1.0, (227, 227), MODEL_MEAN_VALUES, swapRB=False)
        genderNet.setInput(blob)
        genderPreds = genderNet.forward()
        gender = genderList[genderPreds[0].argmax()]
        print("Gender : {}, conf = {:.3f}".format(gender, genderPreds[0].max()))

        ageNet.setInput(blob)
        agePreds = ageNet.forward()
        age = ageList[agePreds[0].argmax()]
        print("Age Output : {}".format(agePreds))
        print("Age : {}, conf = {:.3f}".format(age, agePreds[0].max()))

        label = "{},{}".format(gender, age)
        cv.putText(frameFace, label, (bbox[0], bbox[1]-10), cv.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2, cv.LINE_AA)
    return frameFace

In [19]:
def Read_Video(name):
    video=cv.VideoCapture(name)
    ret, frame = video.read()
    frame_height, frame_width= frame.shape[:2]
    out = cv.VideoWriter('output.avi',cv.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))
    print("Processing Video...")
    while cv.waitKey(1)<0 :
        hasFrame,frame=video.read()
        
        if not hasFrame:
            cv.waitKey()
            break
        output = age_gender_detector(frame)
        out.write(output)    
    out.release()
    video.release()
    print("Done processing video")



In [20]:
Read_Video(0);

Processing Video...
Gender : Male, conf = 1.000
Age Output : [[1.9618111e-07 7.8608650e-08 1.4877715e-04 1.7682615e-03 5.2051980e-02
  9.4599724e-01 1.8928265e-05 1.4556728e-05]]
Age : (38-43), conf = 0.946
Gender : Male, conf = 1.000
Age Output : [[1.4760396e-07 1.1127524e-07 3.9406263e-04 4.2850003e-03 1.1758111e-01
  8.7770605e-01 1.4575417e-05 1.8972816e-05]]
Age : (38-43), conf = 0.878
Gender : Male, conf = 1.000
Age Output : [[2.5370014e-07 6.5606804e-07 2.4557440e-03 2.2696408e-02 4.4213638e-01
  5.3260899e-01 4.0645733e-05 6.0934537e-05]]
Age : (38-43), conf = 0.533
Gender : Male, conf = 1.000
Age Output : [[1.8045503e-07 2.4779297e-07 7.6802203e-04 7.0392843e-03 4.6628201e-01
  5.2584952e-01 2.4693996e-05 3.6031699e-05]]
Age : (38-43), conf = 0.526
Gender : Male, conf = 1.000
Age Output : [[6.9693272e-07 4.7757732e-07 1.6715599e-04 2.5776362e-03 3.4389877e-01
  6.5328020e-01 4.9289982e-05 2.5757787e-05]]
Age : (38-43), conf = 0.653
Gender : Male, conf = 1.000
Age Output : [[3.

Gender : Male, conf = 1.000
Age Output : [[5.8453978e-08 2.3775202e-07 2.7666643e-04 2.6906011e-04 8.6794502e-01
  1.3144062e-01 3.7648682e-05 3.0778025e-05]]
Age : (25-32), conf = 0.868
Gender : Male, conf = 1.000
Age Output : [[8.8000718e-08 2.4205681e-07 6.6068844e-04 5.8493181e-04 3.5221633e-01
  6.4634794e-01 8.3101528e-05 1.0665097e-04]]
Age : (38-43), conf = 0.646
Gender : Male, conf = 1.000
Age Output : [[1.1273416e-07 1.2705608e-06 3.5948092e-03 1.2247986e-03 4.2175582e-01
  5.7323700e-01 9.8996396e-05 8.7166954e-05]]
Age : (38-43), conf = 0.573
Gender : Male, conf = 1.000
Age Output : [[1.7226736e-07 5.9756348e-07 8.3246088e-04 6.3879089e-04 1.6153067e-01
  8.3690828e-01 4.7249792e-05 4.1750878e-05]]
Age : (38-43), conf = 0.837
Gender : Male, conf = 1.000
Age Output : [[4.3199723e-08 7.6223145e-08 1.7590511e-04 1.3053506e-04 3.8197923e-02
  9.6141362e-01 5.2446161e-05 2.9325651e-05]]
Age : (38-43), conf = 0.961
Gender : Male, conf = 1.000
Age Output : [[8.7677826e-08 3.708945

Gender : Male, conf = 1.000
Age Output : [[4.6225022e-08 1.2160189e-08 3.3449109e-05 6.0269573e-05 1.1145042e-01
  8.8842082e-01 2.5185052e-05 9.7771908e-06]]
Age : (38-43), conf = 0.888
Gender : Male, conf = 1.000
Age Output : [[2.3997631e-08 6.7459518e-09 2.4904555e-05 7.2850555e-05 1.4480035e-01
  8.5506988e-01 1.7239076e-05 1.4755192e-05]]
Age : (38-43), conf = 0.855
Gender : Male, conf = 1.000
Age Output : [[5.9710111e-08 4.7098300e-08 1.4877929e-04 2.3764391e-04 4.1283718e-01
  5.8670574e-01 3.0881733e-05 3.9665378e-05]]
Age : (38-43), conf = 0.587
Gender : Male, conf = 1.000
Age Output : [[3.2927801e-08 2.0833436e-08 1.2853704e-04 2.4319853e-04 1.3037041e-01
  8.6920619e-01 1.9584042e-05 3.2006479e-05]]
Age : (38-43), conf = 0.869
Gender : Male, conf = 1.000
Age Output : [[1.1809690e-07 9.2572428e-08 1.6599095e-04 6.9659535e-04 2.4317108e-01
  7.5588763e-01 3.2323282e-05 4.6154288e-05]]
Age : (38-43), conf = 0.756
Gender : Male, conf = 1.000
Age Output : [[1.2164311e-07 6.563819

Gender : Male, conf = 1.000
Age Output : [[1.9584377e-07 4.0757659e-08 2.4923487e-05 1.6161123e-04 1.0575037e-02
  9.8919934e-01 2.8427436e-05 1.0480953e-05]]
Age : (38-43), conf = 0.989
Gender : Male, conf = 1.000
Age Output : [[1.2985002e-07 4.4108941e-08 6.1401690e-05 2.6550773e-04 4.0573500e-02
  9.5906764e-01 2.2199210e-05 9.5632440e-06]]
Age : (38-43), conf = 0.959
Gender : Male, conf = 1.000
Age Output : [[1.6089191e-07 4.6986916e-08 8.1478349e-05 2.6484646e-04 3.0898970e-02
  9.6871692e-01 2.6691463e-05 1.0861583e-05]]
Age : (38-43), conf = 0.969
Gender : Male, conf = 1.000
Age Output : [[1.4098958e-07 4.8090765e-08 5.7420297e-05 5.0874334e-04 1.7079824e-01
  8.2860088e-01 2.3453993e-05 1.1094794e-05]]
Age : (38-43), conf = 0.829
Gender : Male, conf = 1.000
Age Output : [[1.6704519e-07 7.9690338e-08 1.3905675e-04 8.8204624e-04 1.5804175e-01
  8.4090638e-01 1.8769439e-05 1.1754151e-05]]
Age : (38-43), conf = 0.841
Gender : Male, conf = 1.000
Age Output : [[9.0118995e-08 2.616800

Gender : Male, conf = 1.000
Age Output : [[1.2582295e-07 2.2880482e-08 2.2843886e-05 2.3264427e-05 1.1053183e-02
  9.8885649e-01 3.2894692e-05 1.1254095e-05]]
Age : (38-43), conf = 0.989
Gender : Male, conf = 1.000
Age Output : [[1.6492567e-07 5.0854556e-08 3.0206729e-05 2.0491330e-04 1.2973480e-01
  8.6998260e-01 3.3652184e-05 1.3687810e-05]]
Age : (38-43), conf = 0.870
Gender : Male, conf = 1.000
Age Output : [[9.5586188e-08 5.0147221e-08 8.9879453e-05 3.4371152e-04 2.7139011e-01
  7.2814584e-01 2.0810894e-05 9.4647803e-06]]
Age : (38-43), conf = 0.728
Gender : Male, conf = 1.000
Age Output : [[5.1075464e-08 1.5687187e-08 4.0475228e-05 3.5299156e-05 6.1930478e-02
  9.3795145e-01 3.2250933e-05 1.0009537e-05]]
Age : (38-43), conf = 0.938
Gender : Male, conf = 1.000
Age Output : [[1.3285820e-07 4.8970698e-08 4.6906069e-05 2.0012312e-04 1.2241421e-01
  8.7728333e-01 4.2297008e-05 1.2990933e-05]]
Age : (38-43), conf = 0.877
Gender : Male, conf = 1.000
Age Output : [[5.2707694e-08 2.955934

KeyboardInterrupt: 

In [2]:
def Read_Image(name):
    input = cv.imread(name)
    output = Age_Gender_Pred(input)
    cv.imshow( "img", output)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [5]:
Read_Image("./img/.jpg")

Gender : Male, conf = 1.000
Age Output : [[4.6952887e-04 6.9918274e-03 2.3511361e-02 7.3980623e-01 2.2104034e-01
  6.1373981e-03 1.6484243e-03 3.9505065e-04]]
Age : (15-20), conf = 0.740
